## Introduction

Pour simuler l'ordonnancement des patients au SUA avec un algorithme génétique et pouvoir comparer les résultats avec d'autres algorithmes d'ordonnancement, on pose certaines hypothèses:
- les patients ont tous un CCMU de 2 pour deux raisons: 
  - permet de focaliser l'attention sur la **réduction du temps d'attente primaire**)
  - modèle assez réaliste: 80% des patients leur est affecté un CCMU de 2 (selon l'analyse des résultats de la base de données médicale au CHU de Lille)

- chaque patient lui sont affectés un nombre de capacités(skills) nécessaires à son traitement au sein du SUA

- pour **pallier au problème de l'interchangeabilité** de certaines opérations, on ordonne les capacités à partir de celles qui s'effectuent en premier jusqu'à celles dont l'ordre est moins important
_
> Ainsi, le soin nécessitant la capacité 1 (une prise de sang par exemple ...) passe toujours avant le soin qui nécessite la capacité 2 (une IRM ou radiologie par exemple) et ainsi de suite ...



**NB**: En vue de s'intéresser uniquement à la comparaison des résultats d'ordonnancement, on opte pour un ***ordonnancement statique*** (la dynamicité sera introduite dans le modèle SMA).

Autrement dit:
-  un nombre de patients "nb_patients" arrivant tous d'un seul coup (*avec un temps d'attente initiale affecté aléatoirement représentant le temps d'attente durant l'enregistrement administratif ou le premier diagnostic*) 

- l'agent ordonnanceur devra s'occuper d'ordonnancer toutes leurs tâches de soins. 

## Paramètres généraux 

On définit un **nombre de patients** dans la zone d'attente, un **nombre de capacités** disponible au sein du SUA et un **temps de traitement** pour chaque tâche de soin nécessitant l'intervention d'une certaine capacité.

**NB:** On choisit un nombre de patients limité à 20 se retrouvant tous dans la salle d'attente au même temps, en train d'attendre leur ordonnancement (situation qui correspond assez bien au cas réel).  

Ce nombre est choisi afin de permetttre de mieux percevoir les résultats des différentes approches qui seront développées dans ce Notebook.

In [1]:
import random as rd

# mise en situation
nb_patients = 20                            # nombre de patients dans la salle d'attente arrivés tous au même temps
nb_capacités = 6
temps_capacité = [20, 40, 30, 50, 30, 20]   # temps nécessaire de traitement pour chaque capacité disponible

Ensuite, on définira la classe Patient avec les capacités nécessaires à son traitement et son temps d'attente initial au cours de l'enregistrement/diagnostic primaire

In [ ]:
class Patient:
  def __init__(self, id):
    self.id = id
    self.attente = rd.randint(0,40)  # temps d'attente initial durant l'enregistrement/le diagnostic
    self.nb_thérapies = rd.randint(1, 4)  # un nombre aléatoire de thérapies à effectuer
    self.taches = rd.sample(range(0, nb_capacités), self.nb_thérapies)   # liste de tâches de soins


On définit ainsi tous les patients dans la zone d'attente

In [ ]:
salle_attente = [Patient(i) for i in range(nb_patients)]

En fonction des tâches de soins nécessaires pour chaque patient, on définit pour chaque capacité la liste des patients à traiter

In [ ]:
# on définit le passage des patients par chaque caapcité
passage_par_capacité = [[] for i in range(nb_capacités)]
for patient in salle_attente:
  for tache in patient.taches:
    passage_par_capacité[tache].append(patient.id)

print(passage_par_capacité)


[[2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 19], [0, 1, 2, 4, 5, 6, 9, 13, 15, 16], [1, 2, 3, 5, 8, 10, 12, 13, 15, 17, 18], [3, 5, 10, 15, 16], [1, 2, 4, 8, 10, 15, 16, 17], [1, 6, 7, 9, 12, 14, 16]]


## Algorithme génétique : première approche 

Les paramètres de l'AG

In [ ]:
# la taille de la population
pop_size = 8 

# le nombre maximal de générations
generation_limit = 100  

# le score maximal de fitness (ici, il représente le temps d'attente primaire moyen des patients en minutes)
fitness_limit = 140   

Génération du chromosome et de la population

In [ ]:
def generate_chromosome():
  """ 
  Le chromosome est une liste en 2D, chaque ligne représente l'ordre de passage des patients par capacité
  """
  chromosome = [rd.sample(passage_par_capacité[i], len(passage_par_capacité[i])) for i in range(nb_capacités)]
  return chromosome


def generate_population():
  """
  La population est une liste de pop_size chromosomes
  """
  return [generate_chromosome() for _ in range(pop_size)]

Le chromosome seul ne permet pas d'avoir une idée sur le temps d'attente des patients avant leur première consultation.

On cherche alors à présenter le chromosome sous forme de schedule par patient.

In [ ]:
def scheduling(chromosome):
  """
  Permet d'ordonnancer les patients suivant leur ordre au sein du chromosome
  Returns
  -------
  schedule : une liste de tuples  : [ [( , , ), ( , , ), ( , , )],
                                      [( , , ), ( , , ), ( , , )], 
                                      [( , , ), ( , , ), ( , , )]  ]
   - Chaque ligne représente l'ordre des thérapies d'un patient précis
   - ( , , ) : la 1ère entrée représente l'indice de la capacité nécessaire, 2ème entrée temps 
  de début de la thérapie et la 3ème temps de fin                
  """

  schedule = [[] for _ in range(nb_patients)]  # chaque ligne représente un patient
  attente_primaire = [patient.attente for patient in salle_attente]
  

  # on représente la disponibilité de chaque capacité en temps réel.
  dispo_capacités = [0 for i in range(nb_capacités)] # toutes les capacités sont disponibles à l'instant 0

  for capacité in range(len(chromosome)):
    list_patients = chromosome[capacité]
    if list_patients:
      for patient in list_patients:
        taches_precedentes = schedule[patient]

        # si le patient a déjà des tâches de soin à subir avant la tâche actuelle
        if taches_precedentes:
          last_task = taches_precedentes[-1]  # last_task est donnée sous forme d'un tuple (capacité, start, end)
          last_end = last_task[2]
          # on vérifie également la disponibilité de la capacité nécessaire à la tâche de soin
          if dispo_capacités[capacité] > last_end:
            new_start = dispo_capacités[capacité]
          else: new_start = last_end
          

        else:
          # c'est la première tâche de soin du patient 
          new_start = dispo_capacités[capacité]
          attente_primaire[patient]+= new_start  #actualiser le temps d'attente primaire du patient
          

        new_end = new_start + temps_capacité[capacité]  # on ajoute le temps de traitement nécessaire
        new_task = (capacité, new_start, new_end)
        dispo_capacités[capacité] = new_end  # la capacité sera disponible après la fin de la tâche de soin actuelle
        schedule[patient].append(new_task)
        

  # on retourne le temps d'attente primaire 
  return schedule, attente_primaire

On définit la fonction "fitness" qui permet de calculer le temps d'**attente primaire** moyen des patients 

In [ ]:
def fitness(chromosome):
  """  permet de calculer le temps d'attente primaire moyen des patients"""
  schedule, attente_primaire = scheduling(chromosome)
  total_attente = 0
  for patient in range(len(attente_primaire)):
    total_attente += attente_primaire[patient]

  attente_moyenne = total_attente/nb_patients
  return attente_moyenne


On définit par la suite les opérations nécessaires à l'algorithme génétique:
- la sélection 
- le croisement
- la mutation

In [ ]:
def selection_pair(population):
      """Selection d'une paire de parents selon le meilleur score fitness"""
      return rd.choices(population, weights = [fitness(chromosome) for chromosome in population], k = 2)

def crossover(chromosomeA, chromosomeB):
    # on choisit deux capacités sui subiront le croisement
    cap = rd.sample(range(nb_capacités), 2)
    cap.sort()
    index1, index2 = cap[0], cap[1]
    # on génère les deux nouveaux chromosomes
    new_A = chromosomeA[0:index1] + [chromosomeB[index1]] + chromosomeA[index1+1:index2] + [chromosomeB[index2]] + chromosomeA[index2+1 :]
    new_B = chromosomeB[0:index1] + [chromosomeA[index1]] + chromosomeB[index1+1:index2] + [chromosomeA[index2]] + chromosomeB[index2+1 :]
    #chromosomeA[index1], chromosomeB[index1] = chromosomeB[index1], chromosomeA[index1]  # deep copy
    #chromosomeA[index2], chromosomeB[index2] = chromosomeB[index2], chromosomeA[index2]
    return new_A, new_B 

def mutation(chromosome):
    # choix aléatoire d'une capacité de soin
    capacité = rd.randint(0, nb_capacités-1)
    while len(chromosome[capacité]) < 2:
      capacité = rd.randint(0, nb_capacités-1)

    # interchanger deux tâches de soins faisant intervenir la capacité sélectionnée
    oper_index = rd.sample(range(len(chromosome[capacité])), 2)  # indices des opérations à interchanger
    chromosome[capacité][oper_index[0]], chromosome[capacité][oper_index[1]] = chromosome[capacité][oper_index[1]], chromosome[capacité][oper_index[0]]
    return chromosome

Finalement, on a alors besoin de chercher la solution optimal à partir d'une solution initiale aléatoire.



In [ ]:
def run_evolution():
  population = generate_population()

  for generation in range(generation_limit):
    # la solution ayant le plus grand score fitness est en position 0
    population = sorted (population, key = lambda chromosome: fitness(chromosome), reverse = False)

    if fitness(population[0])<= fitness_limit:
      break

    next_generation = population[0:2] # on garde les deux premières meilleures solutions

    for pair in range(int(pop_size/2)-1):
      parents = selection_pair(population)
      childA, childB = crossover(parents[0], parents[1])
      childA, childB = mutation(childA), mutation(childB)
      next_generation += [childA, childB]
    
    population = next_generation

  # a la sortie de la boucle
  population = sorted (population, key = lambda chromosome: fitness(chromosome), reverse = False)
  best_solution = population[0]
  return best_solution, generation



Partir d'une solution aléatoire n'est pas forcément optimal. Pour résoudre ce problème, on essaye de relancer l'algorithme plusieurs fois (afin de partir de plusieurs solutions initiales) .

In [ ]:
def best_results():
  best_solution, best_nb_generation = run_evolution()
  best_fitness = fitness(best_solution)

  # in relance l'algo pour 30 itérations
  iterations = 120

  for i in range(iterations):
    solution, generation = run_evolution()
    if fitness(solution)< best_fitness:
      best_solution, best_fitness, best_nb_generation = solution, fitness(solution), generation

  print(best_fitness, best_nb_generation)
  return best_solution
best_solution = best_results()    

116.95 20


La solution optimale produit un temps d'attente primaire moyen entre 100 minutes (**1h 40minutes**) et 130 minutes (**2h 10min**)

**Diagramme de Gantt**

Pour produire le diagramme de gantt de l'ordonnancement optimal, on a d'abord besoin de convertir le temps en format "hh:mm:ss"

In [ ]:
# formattage de temps

def format_time(time):
  heures = 0
  minutes = 0
  while time >= 60:
    heures +=1
    time -= 60
  minutes = time

  if heures < 10:
    heures = "0" + str(heures) +":"
  else:
    heures = str(heures)+":"

  if minutes < 10:
    minutes = "0" + str(minutes) +":00"
  else:
    minutes = str(minutes)+":00"
  return heures+minutes

Ensuite, on procède à la génération du Gantt.

In [ ]:
# transformer la solution (qui est un chromosome en 2D) en un schedule
schedule = scheduling(best_solution)[0]


In [ ]:

agenda = [[] for i in range(nb_capacités)]

for patient in range(len(schedule)):
  taches = schedule[patient] # taches de soins du patient
  if taches:
    for tache in taches:
      capacité, debut, fin = tache[0], tache[1], tache[2]
      agenda[capacité].append((patient, debut, fin))

for capacité in range(len(agenda)):
  taches = agenda[capacité]
  if taches:
    # on ordonnance les taches de chaque capacité par temps de début 
    taches = sorted(taches, key = lambda tache:tache[1], reverse = False)  
    agenda[capacité] = taches

print (agenda)


[[(6, 0, 20), (17, 20, 40), (2, 40, 60), (5, 60, 80), (16, 80, 100), (7, 100, 120), (4, 120, 140), (19, 140, 160), (18, 160, 180)], [(11, 0, 40), (13, 40, 80), (4, 140, 180), (5, 180, 220), (9, 220, 260), (6, 260, 300), (18, 300, 340)], [(8, 0, 30), (3, 30, 60), (12, 60, 90), (14, 90, 120), (4, 180, 210), (5, 220, 250), (15, 250, 280), (13, 280, 310)], [(0, 0, 50), (12, 90, 140), (7, 140, 190), (8, 190, 240), (13, 310, 360), (16, 360, 410), (6, 410, 460), (5, 460, 510), (17, 510, 560), (2, 560, 610), (15, 610, 660)], [(0, 50, 80), (1, 80, 110), (10, 110, 140), (17, 560, 590), (15, 660, 690), (11, 690, 720), (7, 720, 750), (2, 750, 780), (16, 780, 810)], [(8, 240, 260), (16, 810, 830), (18, 830, 850), (13, 850, 870), (15, 870, 890), (17, 890, 910), (9, 910, 930), (11, 930, 950), (6, 950, 970), (1, 970, 990)]]


In [ ]:
# import plotly.express as px
import pandas as pd
import plotly.figure_factory as ff
import numpy as nb

data_frame = []

for capacité in range(len(agenda)):
  taches = agenda[capacité]
  if taches:
    for tache in taches:
      patient_id, debut, fin = tache[0], format_time(tache[1]), format_time(tache[2])
      data_frame.append(
          dict(Task= "Capacité "+str(capacité+1), Start = '2020-10-22 '+debut, Finish = '2020-10-22 '+fin, Resource = "Patient "+str(patient_id))
      )

# créer une figure
# colors = {'Machine 0': '#ff7f0e','Machine 1': '#1f77b4','Machine 2': '#d62728','Machine 3': '#2ca02c'}
import matplotlib.cm as cm
# colors = cm.rainbow(np.linspace(0, 1,modele.last_patient))
number_of_colors = nb_patients

#color = ["#"+''.join([rd.choice('0123456789ABCDEF') for j in range(6)])
             #for i in range(number_of_colors)]

color = ["#"+''.join(rd.sample('0123456789ABCDEF', 6) )
             for i in range(number_of_colors)]
dict_colors = {}
for i in range(number_of_colors):
  dict_colors["Patient "+str(i)] = color[i]

fig = ff.create_gantt(data_frame, colors=dict_colors, index_col='Resource', show_colorbar=True,group_tasks=True)
# fig = create_gantt(data_frame)
fig.show()

Comme l'on peut remarquer, même si l'algorithme permet de **réduire le temps d'attente primaire** des patients, l'ordonnancement n'est pas optimal en terme de **distribution de la charge** des tâches de soins par capacité, ce qui est normal car les capacités sont ordonnées par ordre de priorité de soin.

Autrement dit, un patient qui doit effectuer les opérations nécessitant l'intervention des capacités 1, 4, 5, 6 ne peut pas, par exemple, effectuer la tâche de soin 4 avant d'effectuer la première. 

Cette solution permet de **pallier au problème de la permutabilité** des tâches de soin mais avec un prix à payer: la non-uniformité de la charge des tâches de soins par capacité médicale au sein des SUA.

Ainsi, la **durée totale d'exécution Cmax** est assez longue (**16h**).

## Approche envisageable avec les taches de soins pouvant être permutées

Avec cette approche, on autorise la permutabilité des tâches de soins afin d'optimiser au mieux le temps d'attente primaire des patients.

La différence avec l'approche précédente c'est que, au lieu d'ordonner horizontalement les tâches de soins (capacité par capacité), on ordonnance les tâches verticalement en commencant d'abord par les premières tâches de soins de chaque capacité.

In [ ]:
def scheduling(chromosome):
  """
  Permet d'ordonnancer les patients suivant leur ordre au sein du chromosome
  Returns
  -------
  schedule : une liste de tuples  : [ [( , , ), ( , , ), ( , , )],
                                      [( , , ), ( , , ), ( , , )], 
                                      [( , , ), ( , , ), ( , , )]  ]
   - Chaque ligne représente l'ordre taches de soins d'un certain patient 
   - ( , , ) : la 1ère entrée représente l'id du patient convern&, 2ème entrée temps 
  de début de la thérapie et la 3ème temps de fin                
  """
  
  schedule = [[] for _ in range(nb_patients)]  
  attente_primaire = [patient.attente for patient in salle_attente]

  # on représente la disponibilité de chaque capacité en temps réel.
  dispo_capacités = [0 for i in range(nb_capacités)] # toutes les capacités sont disponibles à l'instant 0

  # on calcule la longueur du la plus grande liste de taches de soins dans le chromosome
  longest = 0
  for capacité in chromosome:
    if len(capacité) >longest: longest=len(capacité)

  compteur = 0
  while compteur < longest:
    for capacité in range(len(chromosome)):
      if len(chromosome[capacité])>=compteur+1 :
        patient = chromosome[capacité][compteur]  # patient actuel
        taches_precedentes = schedule[patient] # les taches de soins déjà programmées pour le patient

        if taches_precedentes:
          # le temps de fin de la dernière tâche de soin programmée
          last_end = taches_precedentes[-1][2]
          if dispo_capacités[capacité] > last_end:
            new_start = dispo_capacités[capacité] 
          else:new_start = last_end

        else:
          # c'est la première tâche de soin 
          new_start = dispo_capacités[capacité] 
          attente_primaire[patient]+= new_start  #actualiser le temps d'attente primaire du patient

        new_end = new_start + temps_capacité[capacité]  # on ajoute le temps de traitement nécessaire
        new_task = (capacité, new_start, new_end)
        dispo_capacités[capacité] = new_end  # la capacité sera disponible après la fin de la tâche de soin actuelle
        schedule[patient].append(new_task)
    compteur+=1

  return schedule, attente_primaire


Comme la dernière fois, on a besoin de déclencher l'ordonnancement à partir de plusieurs solutions initiales .

In [ ]:
def best_results():
  best_solution, best_nb_generation = run_evolution()
  best_fitness = fitness(best_solution)

  # in relance l'algo pour 30 itérations
  iterations = 120

  for i in range(iterations):
    solution, generation = run_evolution()
    if fitness(solution)< best_fitness:
      best_solution, best_fitness, best_nb_generation = solution, fitness(solution), generation

  print(best_fitness, best_nb_generation)
  return best_solution
best_solution = best_results() 

67.95 0


On remarque alors que les résultats sont nettement meilleurs (**moins d'1h d'attente primaire en moyenne**), mais qui vient avec un prix à payer: la permutabilité des tâches de soins des patients.

On reproduit le diagramme de Gantt pour cette solution : 

In [ ]:
# transformer la solution (qui est un chromosome en 2D) en un schedule
schedule = scheduling(best_solution)[0]


In [ ]:
agenda = [[] for i in range(nb_capacités)]

for patient in range(len(schedule)):
  taches = schedule[patient] # taches de soins du patient
  if taches:
    for tache in taches:
      capacité, debut, fin = tache[0], tache[1], tache[2]
      agenda[capacité].append((patient, debut, fin))

for capacité in range(len(agenda)):
  taches = agenda[capacité]
  if taches:
    # on ordonnance les taches de chaque capacité par temps de début 
    taches = sorted(taches, key = lambda tache:tache[1], reverse = False)  
    agenda[capacité] = taches

print (agenda)

[[(6, 0, 20), (19, 20, 40), (18, 40, 60), (16, 60, 80), (7, 80, 100), (5, 180, 200), (2, 300, 320), (4, 320, 340), (17, 340, 360)], [(18, 0, 40), (6, 40, 80), (11, 80, 120), (4, 120, 160), (13, 160, 200), (9, 200, 240), (5, 240, 280)], [(3, 0, 30), (12, 30, 60), (14, 60, 90), (13, 90, 120), (5, 150, 180), (15, 180, 210), (8, 210, 240), (4, 340, 370)], [(13, 0, 50), (0, 50, 100), (5, 100, 150), (17, 150, 200), (7, 200, 250), (2, 250, 300), (12, 300, 350), (16, 350, 400), (8, 400, 450), (15, 450, 500), (6, 500, 550)], [(10, 0, 30), (7, 30, 60), (2, 60, 90), (15, 90, 120), (1, 120, 150), (16, 150, 180), (0, 180, 210), (11, 210, 240), (17, 360, 390)], [(8, 0, 20), (9, 20, 40), (17, 40, 60), (1, 60, 80), (16, 80, 100), (18, 100, 120), (13, 200, 220), (11, 240, 260), (6, 260, 280), (15, 500, 520)]]


In [ ]:
# import plotly.express as px
import pandas as pd
import plotly.figure_factory as ff
import numpy as nb

data_frame = []

for capacité in range(len(agenda)):
  taches = agenda[capacité]
  if taches:
    for tache in taches:
      patient_id, debut, fin = tache[0], format_time(tache[1]), format_time(tache[2])
      data_frame.append(
          dict(Task= "Capacité "+str(capacité+1), Start = '2020-10-22 '+debut, Finish = '2020-10-22 '+fin, Resource = "Patient "+str(patient_id))
      )

# créer une figure
# colors = {'Machine 0': '#ff7f0e','Machine 1': '#1f77b4','Machine 2': '#d62728','Machine 3': '#2ca02c'}
import matplotlib.cm as cm
# colors = cm.rainbow(np.linspace(0, 1,modele.last_patient))
number_of_colors = nb_patients

#color = ["#"+''.join([rd.choice('0123456789ABCDEF') for j in range(6)])
             #for i in range(number_of_colors)]

color = ["#"+''.join(rd.sample('0123456789ABCDEF', 6) )
             for i in range(number_of_colors)]
dict_colors = {}
for i in range(number_of_colors):
  dict_colors["Patient "+str(i)] = color[i]

fig = ff.create_gantt(data_frame, colors=dict_colors, index_col='Resource', show_colorbar=True,group_tasks=True)
# fig = create_gantt(data_frame)
fig.show()

Autoriser la condition de permutabilité des tâches de soins permet à la fois de réduire le temps d'attente primaire des patients (**1h**) mais également la durée maximale d'exécution *Cmax* (**8h**).

Pourtant cette solution n'est pas très pratique en réalité car les patients ont souvent des tâches de soins à subir en priorité par rapport à d'autres.

## Approche pour améliorer le premier modèle (sans permuter les tâches de soins des patients)

Une alternative pour améliorer la première solution présentée sans avoir besoin à payer comme prix la permutabilité des tâches de soin (ce qui n'est pas pratique dans la vie réelle) est d'évaluer les solutions d'ordonnancement sur la base d'un score calculé à partir du  ***temps d'attente globale*** moyen des patients  (= temps d'attente primaire + le temps d'attente entre les différentes tâches de soins) ET du ***temps d'attente primaire*** moyen des patients, et ce suivant la formule suivante:
- score = moyenne_primaire*2 + moyenne_globale

Ainsi, il suffit alors de prendre en compte ce score au lieu de leur attente primaire.

In [ ]:
# on a besoin de redéfinir la fitness_limit, car cette fois-ci il s'agit bien de calculer un score (non pas un temps d'attente simple)
fitness_limit = 680

On garde le même type d'ordonnancement que dans la première approche (pour favoriser l'ordre des tâches de soin sans permutation).

In [ ]:
def scheduling(chromosome):
  """
  Permet d'ordonnancer les patients suivant leur ordre au sein du chromosome
  Returns
  -------
  schedule : une liste de tuples  : [ [( , , ), ( , , ), ( , , )],
                                      [( , , ), ( , , ), ( , , )], 
                                      [( , , ), ( , , ), ( , , )]  ]
   - Chaque ligne représente l'ordre des thérapies d'un patient précis
   - ( , , ) : la 1ère entrée représente l'indice de la capacité nécessaire, 2ème entrée temps 
  de début de la thérapie et la 3ème temps de fin                
  """

  schedule = [[] for _ in range(nb_patients)]  # chaque ligne représente un patient
  attente_primaire = [patient.attente for patient in salle_attente]
  attente_globale = [patient.attente for patient in salle_attente]

  # on représente la disponibilité de chaque capacité en temps réel.
  dispo_capacités = [0 for i in range(nb_capacités)] # toutes les capacités sont disponibles à l'instant 0

  for capacité in range(len(chromosome)):
    list_patients = chromosome[capacité]
    if list_patients:
      for patient in list_patients:
        taches_precedentes = schedule[patient]

        # si le patient a déjà des tâches de soin à subir avant la tâche actuelle
        if taches_precedentes:
          last_task = taches_precedentes[-1]  # last_task est donnée sous forme d'un tuple (capacité, start, end)
          last_end = last_task[2]
          # on vérifie également la disponibilité de la capacité nécessaire à la tâche de soin
          if dispo_capacités[capacité] > last_end:
            new_start = dispo_capacités[capacité]
          else: new_start = last_end
          attente_globale[patient] += new_start - last_end

        else:
          # c'est la première tâche de soin du patient 
          new_start = dispo_capacités[capacité]
          attente_primaire[patient]+= new_start  #actualiser le temps d'attente primaire du patient
          attente_globale[patient] += new_start 

        new_end = new_start + temps_capacité[capacité]  # on ajoute le temps de traitement nécessaire
        new_task = (capacité, new_start, new_end)
        dispo_capacités[capacité] = new_end  # la capacité sera disponible après la fin de la tâche de soin actuelle
        schedule[patient].append(new_task)
        

  # on retourne le temps d'attente primaire 
  return schedule, attente_globale, attente_primaire

In [ ]:
def fitness(chromosome):
  """  permet de calculer un score fitness en se basant à la fois sur le temps d'attente primaire mais aussi le temps globale
  des patients au SUA"""
  schedule, attente_globale, attente_primaire = scheduling(chromosome)
  moyenne_primaire = 0
  moyenne_globale = 0
  for patient in range(len(attente_globale)):
    moyenne_globale += attente_globale[patient]
    moyenne_primaire += attente_primaire[patient]

  moyenne_primaire /= nb_patients
  moyenne_globale /= nb_patients

  # calcul du score fitness 
  score = moyenne_primaire*2 + moyenne_globale
  return score

In [ ]:
def best_results():
  best_solution, best_nb_generation = run_evolution()
  best_fitness = fitness(best_solution)

  # in relance l'algo pour 30 itérations
  iterations = 120

  for i in range(iterations):
    solution, generation = run_evolution()
    if fitness(solution)< best_fitness:
      best_solution, best_fitness, best_nb_generation = solution, fitness(solution), generation

  print(best_fitness, best_nb_generation)
  return best_solution
best_solution = best_results() 

523.75 20


In [ ]:
# transformer la solution (qui est un chromosome en 2D) en un schedule
schedule, attente_globale, attente_primaire = scheduling(best_solution)


On affiche le temps d'attente primaire moyen et le temps globale moyen des patients au SUA donnés à partir de la meilleure solution trouvée précédemment. 

In [ ]:
moyenne_globale = 0
moyenne_primaire = 0

for i in range(len(attente_globale)):
  moyenne_globale += attente_globale[i]
  moyenne_primaire += attente_primaire[i]

moyenne_globale /= nb_patients
moyenne_primaire /= nb_patients

print(f"temps d'attente primaire moyen des patients est : {format_time(moyenne_primaire)} \n")
print(f"temps d'attente globale moyen des patients est : {format_time(moyenne_globale)} \n")

temps d'attente primaire moyen des patients est : 01:42.75:00 

temps d'attente globale moyen des patients est : 05:18.25:00 



Etant donné que le score fitness prend en compte cette fois-ci le ***temps d'attente primaire*** ET ***globale des patients***, cela permet de trouver une solution optimale entre les deux. 

In [ ]:
agenda = [[] for i in range(nb_capacités)]

for patient in range(len(schedule)):
  taches = schedule[patient] # taches de soins du patient
  if taches:
    for tache in taches:
      capacité, debut, fin = tache[0], tache[1], tache[2]
      agenda[capacité].append((patient, debut, fin))

for capacité in range(len(agenda)):
  taches = agenda[capacité]
  if taches:
    # on ordonnance les taches de chaque capacité par temps de début 
    taches = sorted(taches, key = lambda tache:tache[1], reverse = False)  
    agenda[capacité] = taches

print (agenda)

[[(18, 0, 20), (2, 20, 40), (6, 40, 60), (16, 60, 80), (7, 80, 100), (19, 100, 120), (5, 120, 140), (4, 140, 160), (17, 160, 180)], [(13, 0, 40), (18, 40, 80), (9, 80, 120), (11, 120, 160), (4, 160, 200), (5, 200, 240), (6, 240, 280)], [(3, 0, 30), (13, 40, 70), (12, 70, 100), (8, 100, 130), (15, 130, 160), (4, 200, 230), (5, 240, 270), (14, 270, 300)], [(0, 0, 50), (2, 50, 100), (13, 100, 150), (6, 280, 330), (17, 330, 380), (12, 380, 430), (16, 430, 480), (5, 480, 530), (7, 530, 580), (15, 580, 630), (8, 630, 680)], [(1, 0, 30), (11, 160, 190), (2, 190, 220), (10, 220, 250), (0, 250, 280), (16, 480, 510), (17, 510, 540), (15, 630, 660), (7, 660, 690)], [(13, 150, 170), (18, 170, 190), (6, 330, 350), (9, 350, 370), (1, 370, 390), (16, 510, 530), (17, 540, 560), (15, 660, 680), (8, 680, 700), (11, 700, 720)]]


In [ ]:
# import plotly.express as px
import pandas as pd
import plotly.figure_factory as ff
import numpy as nb

data_frame = []

for capacité in range(len(agenda)):
  taches = agenda[capacité]
  if taches:
    for tache in taches:
      patient_id, debut, fin = tache[0], format_time(tache[1]), format_time(tache[2])
      data_frame.append(
          dict(Task= "Capacité "+str(capacité+1), Start = '2020-10-22 '+debut, Finish = '2020-10-22 '+fin, Resource = "Patient "+str(patient_id))
      )

# créer une figure
# colors = {'Machine 0': '#ff7f0e','Machine 1': '#1f77b4','Machine 2': '#d62728','Machine 3': '#2ca02c'}
import matplotlib.cm as cm
# colors = cm.rainbow(np.linspace(0, 1,modele.last_patient))
number_of_colors = nb_patients

#color = ["#"+''.join([rd.choice('0123456789ABCDEF') for j in range(6)])
             #for i in range(number_of_colors)]

color = ["#"+''.join(rd.sample('0123456789ABCDEF', 6) )
             for i in range(number_of_colors)]
dict_colors = {}
for i in range(number_of_colors):
  dict_colors["Patient "+str(i)] = color[i]

fig = ff.create_gantt(data_frame, colors=dict_colors, index_col='Resource', show_colorbar=True,group_tasks=True)
# fig = create_gantt(data_frame)
fig.show()

Cette approche permet de trouver un compromis entre les deux dernières apporches : une approche qui permet de trouver une optimalité à la fois du ***temps d'attente primaire*** des patients, du ***temps d'attente globale*** au SUA et qui permet en plus de diminuer la ***durée d'exécution des tâches de soins Cmax*** au SUA à **12h** (contre **16h** avec la prmière approche). De plus, cela étant donné la non-permutabilité des tâches de soins des patients. 

## Conclusion

Pour notre ordonnancement à l'aide de l'algorithme génétique, on a essayé trois modèles:
- **1er modèle** : il respecte bien la priorité des tâches de soins (si par exemple une IRM passe d'abord avant une prise de sang). Il permet de *pallier au problème de permutabilité des tâches de soin*. Par contre, la distribution de la charge des tâches de soins par capacité n'est pas optimal.

- **2ème modèle** : un modèle qui s'adapte bien avec les tâches de soins d'un même patient pouvant être permutée sans problème. Il permet de donner un *meilleur résultat en terme de temps d'attente primaire* moyen des patients, mais par contre la solution n'est pas très pratique dans la vie réelle.

- **3ème modèle** : un modèle qui trouve un compromis entre les deux derniers, avec une solution qui s'adapte au problème d'ordonnancement avec non-permutabilité des tâches de soins d'un même patient, permettant de trouver ***un point optimal entre temps d'attente primaire des patients, leurs temps d'attente globale au SUA ET durée d'exécution totale des tâches de soins Cmax***. . 
